# Tests and sanity checks of HMM implementations

For basic functionality tests of our HMM implementations, we use a simple Markov chain setup based on a coin-flipping task.

Consider a coin-flipping task where there are 2 states: 1) we are holding an unbiased coin and 2) we are holding a biased coin. Between coin flips, there is a probability p that we switch coins. This gives a Markov chain with a 2x2 transition matrix. The unbiased coin state has index 0 and the biased coin state has index 1.
    
There are two possible observations/emissions from each coin flip. We can either observe: 1) heads or 2) tails. The probability of observing heads/tails from the unbiased coin state is 0.5, and the probability from the biased coin state is q/1-q, for some probability q. The heads emission has index 0 and the tails emission has index 1.

In [6]:
import os
import numpy as np

In [2]:
def make_transition_matrix(p):
    return np.array([[p, 1-p], [1-p, p]])

def make_emission_matrix(q):
    return np.array([[0.5, 0.5], [q, 1-q]])

def draw_sequence(A, B, prior, T):
    num_states, num_obs = B.shape
    state = np.random.choice(num_states, 1, p=prior).item()
    emissions = [np.random.choice(num_obs, 1, p=B[state].flatten()).item()]
    for t in range(1, T):
        state = np.random.choice(num_states, 1, p=A[state].flatten()).item()
        emissions.append(np.random.choice(num_obs, 1, p=B[state].flatten()).item())
    return emissions

## Multinomial HMM based on 1st order Markov chain


In [3]:
from uncertainty_motion_prediction.predictor import HMMMultinomialFirstOrder

We test the forward/backward algorithms used for estimating the likelihood of a given sequence of emissions/observations.

In [4]:
p = 0.5
q = 0.5
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [5]:
hmm1 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm1.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq1 = np.array([0, 0, 1, 0])
prob1 = hmm1.get_sequence_likelihood_backward(test_seq1)
prob2 = hmm1.get_sequence_likelihood(test_seq1)

print(prob1, prob2)

0.0625 0.06250000000000003


We test the decoding algorithms used to estimate the most likely sequence of hidden states that produced the sequence of emissions/observations in our data. The decoding algorithm implemented is the Viterbi algorithm.

In [6]:
p = 0.5
q = 0.999 # Use 0.999 instead of 1, because I have yet to handle the case of np.log(0) in log-sum-exp function.
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [7]:
hmm2 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm2.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq2 = np.array([1, 0, 0])
path, prob = hmm2.decode(test_seq2)
print(path)
print(prob)

[0, 1, 1]
0.06237506250000003


Test the estimation of HMM model parameters from data. This uses the specialised form of the EM algorithm, the Baum-Welch algorithm. The algorithm can be prone to local minima, and may not converge well when all the parameters to be estimated are not well-initialised. You can test its robustness to initialisation but selectively initialising some of the parameters using the actual values.

In [8]:
p = 0.5
q = 0.999 # Use 0.999 instead of 1, because I have yet to handle the case of np.log(0) in log-sum-exp function.
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

In [9]:
data = np.array([draw_sequence(transition_matrix, emission_matrix, prior, 10) for i in range(500)])
print(data)

# Don't give the correct values for any of the parameters. Use random initialisation.
hmm3 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm3.estimate_parameters(data)

print("Transition")
print(np.exp(hmm3._log_A))
print("Emission")
print(np.exp(hmm3._log_B))
print("Prior")
print(np.exp(hmm3._log_phi))

hmm3.save_to_file("./hmm_sanity_check_param.pickle")

hmm3_load = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm3_load.load_from_file("./hmm_sanity_check_param.pickle")
hmm3_load.estimate_parameters(data)
print("Loaded and Trained Transition")
print(np.exp(hmm3_load._log_A))

[[0 0 0 ... 1 1 0]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
Estimating HMM model parameters...
Iter 1, log-likelihood loss: -7.024722607974975, delta: inf
Iter 2, log-likelihood loss: -5.664936102659883, delta: 1.3597865053150917
Iter 3, log-likelihood loss: -5.66131013548803, delta: 0.0036259671718532616
Iter 4, log-likelihood loss: -5.658889304612112, delta: 0.0024208308759181563
Iter 5, log-likelihood loss: -5.657255492287608, delta: 0.0016338123245036584
Iter 6, log-likelihood loss: -5.656145098736711, delta: 0.0011103935508973706
Iter 7, log-likelihood loss: -5.655387159901618, delta: 0.0007579388350924532
Iter 8, log-likelihood loss: -5.65486849272569, delta: 0.0005186671759283001
Iter 9, log-likelihood loss: -5.654513089781826, delta: 0.0003554029438639361
Iter 10, log-likelihood loss: -5.654269421616268, delta: 0.00024366816555776438
Iter 11, log-likelihood loss: -5.654102342983989, delta: 0.00016707863227960473
Iter 1

In [10]:
data = np.array([draw_sequence(transition_matrix, emission_matrix, prior, 10) for i in range(500)])
print(data)

# Initialise the transition and prior models using correct values, and try to estimate the emission probabilities.
hmm3 = HMMMultinomialFirstOrder(2, 2, verbose=True)
hmm3._initialise_transition_model(transition_matrix)
hmm3._initialise_prior_distribution(prior)
hmm3.estimate_parameters(data)

print("Transition")
print(np.exp(hmm3._log_A))
print("Emission")
print(np.exp(hmm3._log_B))
print("Prior")
print(np.exp(hmm3._log_phi))

[[0 0 0 ... 1 1 0]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]]
Estimating HMM model parameters...
Iter 1, log-likelihood loss: -7.3160232371448695, delta: inf
Iter 2, log-likelihood loss: -5.653891440391395, delta: 1.6621317967534743
Iter 3, log-likelihood loss: -5.653879230677302, delta: 1.220971409310323e-05
Iter 4, log-likelihood loss: -5.653867394295645, delta: 1.1836381657026607e-05
Iter 5, log-likelihood loss: -5.653855915128039, delta: 1.1479167605799034e-05
Iter 6, log-likelihood loss: -5.653844777992499, delta: 1.113713554001805e-05
Iter 7, log-likelihood loss: -5.653833968573029, delta: 1.0809419470625414e-05
Iter 8, log-likelihood loss: -5.653823473355083, delta: 1.0495217945383217e-05
Iter 9, log-likelihood loss: -5.65381327956661, delta: 1.0193788472889764e-05
Iter 10, log-likelihood loss: -5.653803375123801, delta: 9.9044428090167e-06
Transition
[[0.45704322 0.54295678]
 [0.45960241 0.54039759]]
Emission
[[0.6838772

## Multinomial HMM based on 2nd order Markov chain

In [7]:
from uncertainty_motion_prediction.predictor import HMMMultinomialSecondOrder

In [8]:
def make_transition_matrix(p):
    return np.array([[[p, 1-p], [1-p, p]], [[p, 1-p], [1-p, p]]])

def make_emission_matrix(q):
    return np.array([[0.5, 0.5], [q, 1-q]])

def draw_sequence(A, B, prior, T):
    num_states, num_obs = B.shape
    state = np.random.choice(num_states, 1, p=prior).item()
    emissions = [np.random.choice(num_obs, 1, p=B[state].flatten()).item()]
    last_state = np.random.choice(num_states, 1, p=prior).item() 
    for t in range(1, T):
        state = np.random.choice(num_states, 1, p=A[last_state][state].flatten()).item()
        emissions.append(np.random.choice(num_obs, 1, p=B[state].flatten()).item())
        last_state = state 
    return emissions

In [16]:
import pickle
from pathlib import Path
import numpy as np
from uncertainty_motion_prediction.predictor import HMMBase

# reference: https://journals.sagepub.com/doi/full/10.1177/1550147718772541

class HMMMultinomialSecondOrder(HMMBase):
    def __init__(self,
                 state_dim,
                 obs_dim,
                 tol=1e-5,
                 max_iters=1,
                 verbose=False):

        # Initialise the dimensions first, so the overriding functions to
        # initialise the transition/observation/initial distributions have
        # access to the dimensions when called from the base class
        self._state_dim = state_dim
        self._obs_dim = obs_dim

        # Initialise the base class
        super().__init__(tol=tol, max_iters=max_iters, verbose=verbose)


    ######################################
    ### HMM model parameter estimation ###
    ######################################

    # Takes in a collection of sequences of the same length, and learns
    # the HMM model from these sequences.
    def estimate_parameters(self, X):
        N, T = X.shape
        prev_log_prob = 0.0
        curr_log_prob = np.inf
        itr = 0

        if self._verbose:
            print("Estimating HMM model parameters...")

        while abs(prev_log_prob - curr_log_prob) > self._tol and itr < self._max_iters:
            itr += 1

            # E-step
            log_gammas = []
            log_xis = []
            average_seq_log_likelihood = 0.0
            for seq in X:
                forward_lattice = self._forward(seq)
                backward_lattice = self._backward(seq)
                seq_log_prob = self._get_sequence_log_likelihood(forward_lattice)

                seq_log_gamma = forward_lattice + backward_lattice - seq_log_prob
                seq_log_xi = np.zeros((self._state_dim, self._state_dim, self._state_dim, T-1))

                for j in range(self._state_dim):
                    for k in range(self._state_dim):
                        for t in range(T-1):
                            for i in range(self._state_dim): 
                                seq_log_xi[i, j, k, t] = forward_lattice[j, t] + \
                                                    self._get_transition_model_log_prob(i, j, k) + \
                                                    self._get_emission_log_prob(k, seq[t+1]) + \
                                                    backward_lattice[j, t+1] - \
                                                    seq_log_prob

                log_gammas.append(seq_log_gamma)
                log_xis.append(seq_log_xi)
                average_seq_log_likelihood += seq_log_prob

            log_gammas = np.array(log_gammas)
            log_xis = np.array(log_xis)

            # M-step
            self._update_transition_model(log_xis)
            self._update_prior(log_gammas)
            self._update_emission_model(log_gammas, X)

            # Update counters
            prev_log_prob = curr_log_prob
            curr_log_prob = average_seq_log_likelihood / float(N)
            if self._verbose:
                print(f'Iter {itr}, log-likelihood loss: {curr_log_prob}, delta: {abs(prev_log_prob - curr_log_prob)}')


    #######################################################################
    ### Sequence likelihood estimation, forward and backward algorithms ###
    #######################################################################

    # Compute the likelihood of seeing the sequence x under 
    # the current HMM model
    def get_sequence_likelihood(self, x):
        lattice = self._forward(x)
        print('forward lattice', np.exp(lattice))
        # print('will be extracting this part only1', np.exp(self._logsumexp(lattice)))
        # print('will be extracting this part only2', np.exp(np.apply_along_axis(self._logsumexp, 0, lattice)))
        log_prob = self._get_sequence_log_likelihood(lattice)
        print('forward log_prob', np.exp(log_prob))
        return np.exp(log_prob)

    # Compute the likelihood of seeing the sequence x under
    # the current HMM model. This method uses the backward
    # algorithm, but since forward/backward are functionally
    # equivalent, the output should be exactly the same
    # as get_sequence_likelihood, up to numerical rounding.
    def get_sequence_likelihood_backward(self, x):
        lattice = self._backward(x)
        print('_backward lattice', np.exp(lattice))
        # log_probs = lattice[:, 0] + self._log_phi + self._get_emission_log_prob_batch(x[0])
        log_probs = lattice[:, :, 0] + self._log_phi + self._get_emission_log_prob_batch(x[0])
        log_prob = self._logsumexp(log_probs)
        return np.exp(log_prob)

    def _get_sequence_log_likelihood(self, forward_lattice):
        return self._logsumexp(forward_lattice[:, :, -1]) # P(O|lambda), probability of observing sequence given parameters

    # Takes in a sequence of length T and computes the alpha values for that sequence
    def _forward(self, x):
        T = len(x)
        dp_table = np.zeros((self._state_dim, self._state_dim, T))
        for j in range(self._state_dim):
            for k in range(self._state_dim):
                dp_table[j, k, 0] = self._log_phi[j] + self._get_emission_log_prob(j, x[0]) 
                dp_table[j, k, 1] = self._log_phi[k] + self._get_emission_log_prob(k, x[1])
        
        for t in range(1, T-1): 
            for j in range(self._state_dim):
                for k in range(self._state_dim):
                    prev_alphas =  dp_table[j, k, t] + dp_table[j, k, t-1]
                    single_transition_log_probs = prev_alphas + self._get_transition_model_log_prob_batch(j, k, dst=True)
                    single_transition_log_probs += self._get_emission_log_prob(j, x[t+1])
                    print('prev_alphas', np.exp(dp_table[j, k, t]), np.exp(dp_table[j, k, t-1]))
                    print('prev_alphas', np.exp(prev_alphas))
                    print('transition', np.exp(self._get_transition_model_log_prob_batch(j, k, dst=True)))
                    print('emission', np.exp(self._get_emission_log_prob(k, x[t+1])))
                    dp_table[j, k, t+1] = self._logsumexp(single_transition_log_probs)
                    print('forward dp_tbl', np.exp(dp_table))

        return dp_table

    # Takes in a sequence of length T and computes the beta values for that sequence
    def _backward(self, x):
        T = len(x)
        dp_table = np.zeros((self._state_dim, self._state_dim, T))

        for t in range(T-2, 0, -1):
            for j in range(self._state_dim): 
                for k in range(self._state_dim):
                    prev_betas = dp_table[j, k, t+1]
                    single_transition_log_probs =  (prev_betas 
                        + self._get_transition_model_log_prob_batch(k, j, dst=False)
                        + self._get_emission_log_prob(k, x[t+1]))
                    dp_table[j, k, t] = self._logsumexp(single_transition_log_probs)
                    print('backward dp_tbl', np.exp(dp_table))

        return dp_table


    ########################
    ### Viterbi decoding ###
    ########################

    # Uses the Viterbi algorithm to find the most likely sequence
    # of hidden states that generates the observations. This sequence
    # of states can be used to make predictions about future states
    # or emissions from the Markov chain.
    def decode(self, x):
        T = len(x)
        log_path_probs = np.zeros((self._state_dim, T))
        backpointers = np.zeros((self._state_dim, T), dtype=np.int32)
        log_path_probs[:, 0] = self._log_phi + self._get_emission_log_prob_batch(x[0])

        for t in range(1, T):
            log_probs_prev_timestep = log_path_probs[:, t-1]
            for state in range(self._state_dim):
                updated_probs = log_probs_prev_timestep + \
                                self._get_transition_model_log_prob_batch(x[t-1], state, dst=True)
                updated_probs += self._get_emission_log_prob(state, x[t])
                log_path_probs[state, t] = np.max(updated_probs)
                backpointers[state, t] = np.argmax(updated_probs)

        best_log_path_prob = np.max(log_path_probs[:, -1])
        best_path_pointer = np.argmax(log_path_probs[:, -1])
        
        best_path = [best_path_pointer]
        for t in range(T-1, 0, -1):
            best_path.append(backpointers[best_path[-1], t])

        return best_path[::-1], np.exp(best_log_path_prob)


    ##################
    ### Prediction ###
    ##################

    # Does greedy prediction of the succeeding states, solely by
    # looking for the transition from the current state that 
    # has the highest probability, moving to the next state and
    # selecting the emission from that state with the highest
    # probability
    def predict_greedy(self, x, N_future):
        best_path, best_path_prob = self.decode(x)
        state = best_path[-1]
        emissions = []
        for n in range(N_future):
            state = np.argmax(self._get_transition_model_log_prob_batch(state, dst=False))
            emissions.append(np.argmax(self._log_B[state, :]))
        return emissions

    # Takes a brute force combinatorial approach to predicting the
    # succeeding states. Does this by enumerating all possible
    # combinations of future states, concatenating them to the
    # current observed sequence of emissions and computing the
    # forward probability over all of them. The sequence of future
    # states with the highest observation probability is selected
    # as the prediction.
    def predict_brute_force(self, x, N_future):
        raise Exception("TODO: Implement this function")


    ################################
    ### HMM model initialisation ###
    ################################

    def _initialise_transition_model_rand(self):
        self._log_A = np.log([[np.random.dirichlet([1 for i in range(self._state_dim)]) for i in range(self._state_dim)] for i in range(self._state_dim)])
        self._log_A = np.array(self._log_A)

    def _initialise_prior_distribution_rand(self):
        self._log_phi = np.log(np.random.dirichlet([1 for i in range(self._state_dim)]).flatten())

    def _initialise_emission_model_rand(self):
        self._log_B = [np.log(np.random.dirichlet([1 for i in range(self._obs_dim)])) for i in range(self._state_dim)]
        self._log_B = np.array(self._log_B)

    def _initialise_transition_model(self, transition_matrix):
        self._log_A = np.log(transition_matrix)

    def _initialise_prior_distribution(self, prior):
        self._log_phi = np.log(prior)

    def _initialise_emission_model(self, emission_matrix):
        self._log_B = np.log(emission_matrix)

    def initialise_parameters(self, transition_matrix, prior, emission_matrix):
        assert(transition_matrix.shape == (self._state_dim, self._state_dim, self._state_dim))
        assert(len(prior) == self._state_dim)
        assert(emission_matrix.shape == (self._state_dim, self._obs_dim))

        self._initialise_transition_model(transition_matrix)
        self._initialise_prior_distribution(prior)
        self._initialise_emission_model(emission_matrix)


    #####################################
    ### Querying HMM model parameters ###
    #####################################

    # Get transition from i --> j
    def _get_transition_model_log_prob(self, i, j, k):
        return self._log_A[i, j, k]
    
    def _get_prior_log_prob(self, state):
        return self._log_phi[state]

    def _get_emission_log_prob(self, state, obs):
        return self._log_B[state, obs]

    # Get vector of log probabilities from the transition model, corresponding
    # to transitions of ALL states --> idx if dst is True, otherwise the
    # transitions of idx --> ALL states if dst is False
    # get i -> j -> k 
    def _get_transition_model_log_prob_batch(self, prev_idx, idx, dst):
        if dst:
            return self._log_A[:, prev_idx, idx]
        else:
            return self._log_A[:, idx, prev_idx]

    # Get the emission probabilities of ALL states on obs
    def _get_emission_log_prob_batch(self, obs):
        return self._log_B[:, obs]


    #####################################
    ### Updating HMM model parameters ###
    #####################################

    # Updates the transition model's log probabilities.
    def _update_transition_model(self, log_xis):
        # log_xis is a 5-d tensor with the dimensions
        # (num seq, num states, num states, num_states, traj len).
        # Each value log_xis[s, i, j, k, t] represents the
        # log transition probability of i --> j --> k at timestep t
        # for the kth sequence. 
        #
        # For each state-state pair, sum the gammas over all timesteps 
        # for all sequences. To transform the sum into a probability
        # measure, normalise by summing over the destination states.
        summed_over_timesteps = np.apply_along_axis(self._logsumexp, 4, log_xis) 
        summed_over_sequences = np.apply_along_axis(self._logsumexp, 0, summed_over_timesteps)
        summed_over_dst_state = np.apply_along_axis(self._logsumexp, 2, summed_over_sequences)
        self._log_A = summed_over_sequences - np.expand_dims(summed_over_dst_state, axis=0).T


    # Updates the prior log probabilities. Effectively finding for each state
    # the expected frequency of being in that state at the first timestep in
    # the sequence. Computed as a log probability to avoid numerical
    # underflow and rounding errors.
    def _update_prior(self, log_gammas):
        # Note that log_gammas is a 3-d tensor with dimensions
        # (num seq, num states, traj len). We will extract the
        # gammas for t = 0 first, then sum over all sequences
        # for each state.
        N = log_gammas.shape[0]
        log_gammas_t0 = log_gammas[:, :, 0]
        self._log_phi = np.apply_along_axis(self._logsumexp, 0, log_gammas_t0) - np.log(N)

    # Updates the emission model's log probabilities.
    def _update_emission_model(self, log_gammas, X):

        # log_gammas is a 3-d tensor with the dimensions
        # (num seq, num states, traj len). We obtain the normalising
        # factor by summing over all data, i.e. both num seq and traj len
        N, T = X.shape
        summed_over_timesteps = np.apply_along_axis(self._logsumexp, 2, log_gammas)
        summed_over_sequences = np.apply_along_axis(self._logsumexp, 0, summed_over_timesteps)

        # Compute the log likelihood of the emission distribution as the
        # expected no. of times we encounter emission vk in state i over
        # the entire dataset
        for state in range(self._state_dim):
            for vk in range(self._obs_dim):
                valid_log_probs = []
                for n in range(N):
                    for t in range(T):
                        if X[n, t] == vk:
                            valid_log_probs.append(log_gammas[n, state, t])

                if len(valid_log_probs) == 0:
                    self._log_B[state, vk] = -np.inf
                else:
                    self._log_B[state, vk] = self._logsumexp(np.array(valid_log_probs))
                
        self._log_B = self._log_B - np.expand_dims(summed_over_sequences, axis=0).T

In [19]:
p = 0.9
q = 0.8
transition_matrix = np.array([[[p, 1-p], [0.5, 0.5]], [[0.5, 0.5], [0.5, 0.5]]])
emission_matrix = np.array([[0.5, 0.5], [q, 1-q]])
prior = np.array([0.5, 0.5])

hmm1 = HMMMultinomialSecondOrder(2, 2, verbose=True)
hmm1.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq1 = np.array([0, 0, 0])

prob1 = hmm1.get_sequence_likelihood_backward(test_seq1)
prob2 = hmm1.get_sequence_likelihood(test_seq1)

print(prob1, prob2)

backward dp_tbl [[[1.  0.7 1. ]
  [1.  1.  1. ]]

 [[1.  1.  1. ]
  [1.  1.  1. ]]]
backward dp_tbl [[[1.   0.7  1.  ]
  [1.   0.48 1.  ]]

 [[1.   1.   1.  ]
  [1.   1.   1.  ]]]
backward dp_tbl [[[1.   0.7  1.  ]
  [1.   0.48 1.  ]]

 [[1.   0.5  1.  ]
  [1.   1.   1.  ]]]
backward dp_tbl [[[1.   0.7  1.  ]
  [1.   0.48 1.  ]]

 [[1.   0.5  1.  ]
  [1.   0.8  1.  ]]]
_backward lattice [[[1.   0.7  1.  ]
  [1.   0.48 1.  ]]

 [[1.   0.5  1.  ]
  [1.   0.8  1.  ]]]
prev_alphas 0.25 0.25
prev_alphas 0.0625
transition [0.9 0.5]
emission 0.5
forward dp_tbl [[[0.25    0.25    0.04375]
  [0.25    0.4     1.     ]]

 [[0.4     0.25    1.     ]
  [0.4     0.4     1.     ]]]
prev_alphas 0.4 0.25
prev_alphas 0.10000000000000002
transition [0.1 0.5]
emission 0.8
forward dp_tbl [[[0.25    0.25    0.04375]
  [0.25    0.4     0.03   ]]

 [[0.4     0.25    1.     ]
  [0.4     0.4     1.     ]]]
prev_alphas 0.25 0.4
prev_alphas 0.10000000000000002
transition [0.5 0.5]
emission 0.5
forward dp_tbl [[[0

In [10]:
p = 0.5
q = 0.999
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])

hmm1.initialise_parameters(transition_matrix, prior, emission_matrix)
test_seq2 = np.array([1, 0, 0, 1])
path, prob = hmm1.decode(test_seq2)
print(path)
print(prob)

[0, 0, 1, 0]
0.12487499999999999


In [11]:
p = 0.5
q = 0.5
transition_matrix = make_transition_matrix(p)
emission_matrix = make_emission_matrix(q)
prior = np.array([0.5, 0.5])
    
data = np.array([draw_sequence(transition_matrix, emission_matrix, prior, 12) for i in range(500)])
print(data)

hmm1 = HMMMultinomialSecondOrder(2, 2, verbose=True)
hmm1._initialise_transition_model(transition_matrix)
hmm1._initialise_prior_distribution(prior)
hmm1.estimate_parameters(data)

print("Transition")
print(np.exp(hmm1._log_A))
print("Emission")
print(np.exp(hmm1._log_B))
print("Prior")
print(np.exp(hmm1._log_phi))

[[1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 1 ... 1 0 1]
 [0 1 1 ... 1 0 1]
 [1 0 1 ... 1 0 1]]
Estimating HMM model parameters...
Iter 1, log-likelihood loss: -7.6726156109352495, delta: inf
Iter 2, log-likelihood loss: -7.615431786708141, delta: 0.057183824227108104
Iter 3, log-likelihood loss: -7.620815414847124, delta: 0.005383628138982743
Iter 4, log-likelihood loss: -7.6192365151072705, delta: 0.0015788997398535898
Iter 5, log-likelihood loss: -7.612326174715188, delta: 0.0069103403920829365
Iter 6, log-likelihood loss: -7.5985124039371845, delta: 0.013813770778003054
Iter 7, log-likelihood loss: -7.581638692056695, delta: 0.01687371188048914
Iter 8, log-likelihood loss: -7.56698867217573, delta: 0.014650019880965814
Iter 9, log-likelihood loss: -7.558699536572356, delta: 0.00828913560337341
Iter 10, log-likelihood loss: -7.558593810143422, delta: 0.00010572642893436068
Iter 11, log-likelihood loss: -7.565966338528428, delta: 0.007372528385006127
Iter 12, 